In [9]:
import os
import numpy as np
import pandas as pd
import sys

module_path_string = "K:\\Github\\GlobalPowerUpdate-Kow\\code\\global_code"
sys.path.append(module_path_string)

import global_function as af

global_path = 'K:\\Github\\GlobalPowerUpdate-Kow\\data\\'

file_path = os.path.join(global_path, 'asia', 'japan')
in_path = os.path.join(file_path, 'raw')
month_path  = os.path.join(in_path, 'month')
out_path_cleaned = af.create_folder(file_path, 'cleaned')
out_path_simulated = af.create_folder(file_path, 'simulated')
in_path_file = os.path.join(in_path, 'occto.csv')
in_path_file_2016 = os.path.join(in_path, 'Japan_daily_generation-2016-v0.1_20201221.csv')
in_path_file_2017 = os.path.join(in_path, 'Japan_daily_generation-2017-v0.1_20201221.csv')
in_path_file_2018 = os.path.join(in_path, 'Japan_daily_generation-demand-2018-0.1_20201221.csv')
# #########################################################
df = pd.read_csv(in_path_file)
file_name = af.search_file(month_path)
df_new = pd.concat(pd.read_csv(f, encoding = 'shift-jis', header = 3) for f in file_name).reset_index(drop = True)
df = pd.concat([df, df_new]).reset_index(drop=True)
# 添加各种日期和单位
df['hour'] = df['時刻'].str.replace('時', '').astype(int)  # 将hour列整理出来
df['date'] = pd.to_datetime(df['月日'])
df['datetime'] = pd.to_datetime(df['date']) + pd.to_timedelta((df['hour']), unit='h')  # 生成datetime

df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['month_date'] = df['datetime'].dt.strftime('%m-%d')

df['unit'] = 'Mwh'
df = df.drop(columns=['月日', '時刻'])

# 改列名为英文
col_list = ['demand', 'nuclear', 'geothermal', 'hydroelectric', 'fossil_fuel_fired', 'biomass', 'wind',
            'wind_regulated', 'photovoltaic',
            'photovoltaic_regulated', 'pumped_storage_hydroelectricity',
            'hour', 'date', 'datetime', 'year', 'month', 'month_date', 'unit']
df.columns = col_list

# 输出
year_list = df['year'].drop_duplicates().tolist()
for x in year_list:
    df[df['year'] == x].to_csv(out_path_cleaned + 'japan-generation-' + str(x) + '-cleaned.csv', index=False,
                               encoding='utf_8_sig')
# ##################################cleaned-simulated######################################################
# iea数据
df_iea = af.iea_data('japan')
result = []
for y in df['year'].drop_duplicates().tolist():
    df_iea_temp = df_iea[df_iea['year'] == y].reset_index(drop=True)
    df_temp = df[df['year'] == y].reset_index(drop=True)
    month_range = df_iea_temp['month'].drop_duplicates().tolist()
    if month_range != 12:
        df_iea_temp = df_iea[df_iea['year'] == y - 1].reset_index(drop=True)
        df_temp = df[df['year'] == y].reset_index(drop=True)
        month_range = df_iea_temp['month'].drop_duplicates().tolist()
    for i in month_range:
        df_temp_monthly = df_temp[df_temp['month'] == i].reset_index(drop=True)

        coal_value = df_iea_temp[df_iea_temp['month'] == i]['coal'].tolist()[0]
        gas_value = df_iea_temp[df_iea_temp['month'] == i]['gas'].tolist()[0]
        oil_value = df_iea_temp[df_iea_temp['month'] == i]['oil'].tolist()[0]

        df_temp_monthly['coal'] = df_temp_monthly['fossil_fuel_fired'] * (
                coal_value / (coal_value + gas_value + oil_value))
        df_temp_monthly['gas'] = df_temp_monthly['fossil_fuel_fired'] * (
                gas_value / (coal_value + gas_value + oil_value))
        df_temp_monthly['oil'] = df_temp_monthly['fossil_fuel_fired'] * (
                oil_value / (coal_value + gas_value + oil_value))

        df_temp_monthly['hydro'] = df_temp_monthly['hydroelectric'] + df_temp_monthly[
            'pumped_storage_hydroelectricity']
        df_temp_monthly['solar'] = df_temp_monthly['photovoltaic'] + df_temp_monthly['photovoltaic_regulated']
        df_temp_monthly['wind'] = df_temp_monthly['wind'] + df_temp_monthly['wind_regulated']
        df_temp_monthly['other'] = df_temp_monthly['biomass'] + df_temp_monthly['geothermal']
        result.append(df_temp_monthly)
df_all = pd.DataFrame(np.concatenate(result), columns=df_temp_monthly.columns)

for x in df_all.columns:
    try:
        df_all[x] = df_all[x].astype(float)
    except:
        pass
af.time_info(df_all, 'datetime')
# for y in df_all['year'].drop_duplicates().tolist():
#     df_hourly = df_all[df_all['year'] == y].reset_index(drop=True)
#     df_daily = df_hourly.copy()
#     df_monthly = df_hourly.copy()
#     out_path_simulated_yearly = af.create_folder(out_path_simulated, str(y))
#     # hourly
#     af.agg(df_hourly, 'datetime', out_path_simulated_yearly, 'hourly',
#            name='Japan_hourly_generation-' + str(y) + '.csv', folder=False, unit=True)
#     # daily
#     df_daily = df_daily.set_index('datetime').resample('d').sum().reset_index()
#     af.agg(df_daily, 'datetime', out_path_simulated_yearly, 'daily',
#            name='Japan_daily_generation-' + str(y) + '.csv', folder=False, unit=True)
#     # monthly
#     df_monthly = df_monthly.set_index('datetime').resample('m').sum().reset_index()
#     af.agg(df_monthly, 'datetime', out_path_simulated_yearly, 'monthly',
#            name='Japan_monthly_generation-' + str(y) + '.csv', folder=False, unit=True)